In [1]:
import polaris as po

In [2]:
dataset = po.load_dataset("asap-discovery/antiviral-potency-2025-unblinded")

Output()

[2025-04-10 13:21:51] WARNING  You're loading data from a remote location. If the dataset is small     ]8;id=163904;file:///Users/cas.wognum/Documents/repositories/polaris/polaris/dataset/_base.py\_base.py]8;;\:]8;id=339491;file:///Users/cas.wognum/Documents/repositories/polaris/polaris/dataset/_base.py#181\181]8;;\
                               enough, consider caching the dataset first using DatasetV2.cache() for              
                               more performant data access.                                                        

[13:21:52]  Success: Fetching dataset                                                                 ]8;id=116769;file:///Users/cas.wognum/Documents/repositories/polaris/polaris/utils/context.py\context.py]8;;\:]8;id=342969;file:///Users/cas.wognum/Documents/repositories/polaris/polaris/utils/context.py#53\53]8;;\

In [3]:
dataset.load_to_memory()

In [4]:
from polaris.dataset import Subset

df = Subset(
    dataset=dataset, 
    indices=list(range(len(dataset))), 
    input_cols=["CXSMILES", "Set"], 
    target_cols=["pIC50 (MERS-CoV Mpro)", "pIC50 (SARS-CoV-2 Mpro)"]
).as_dataframe()

df.head()

,CXSMILES,Set,pIC50 (MERS-CoV Mpro),pIC50 (SARS-CoV-2 Mpro)
0,COC[C@]1(C)C(=O)N(C2=CN=CC3=CC=CC=C23)C(=O)N1C...,Train,4.19,NaN
1,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,Train,4.92,5.29
2,CNC(=O)CN1C[C@]2(C[C@H](C)N(C3=CN=CC=C3C3CC3)C...,Train,4.73,NaN
3,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,Train,4.90,6.11
4,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,Train,4.81,5.62


In [5]:
train = df[df["Set"] == "Train"]
test = df[df["Set"] == "Test"]
len(train), len(test)

(1031, 297)

In [6]:
import json 


with open("../../evaluation/data/exclusions/potency.json") as fd:
    flagged = json.load(fd)

flagged = flagged["pIC50 (SARS-CoV-2 Mpro)"]
flagged = [int(x["idx"]) for x in flagged]
flagged

[5, 8, 188, 194, 275]

In [7]:
import numpy as np


mask = ~np.isin(np.arange(len(test)), flagged)
test = test[mask]
len(test)

292

In [8]:
import pandas as pd


df = pd.concat([train, test]).reset_index(drop=True)
df.head()

,CXSMILES,Set,pIC50 (MERS-CoV Mpro),pIC50 (SARS-CoV-2 Mpro)
0,COC[C@]1(C)C(=O)N(C2=CN=CC3=CC=CC=C23)C(=O)N1C...,Train,4.19,NaN
1,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,Train,4.92,5.29
2,CNC(=O)CN1C[C@]2(C[C@H](C)N(C3=CN=CC=C3C3CC3)C...,Train,4.73,NaN
3,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,Train,4.90,6.11
4,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,Train,4.81,5.62


In [9]:
df.to_csv("../data/potency.csv")

The End.